In [1]:
import pyforest

In [2]:
import nltk #Natural Language tool kit -- this pacakge is quite a mess. Was poorly design and the documentation is not great
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [3]:
df = pd.read_csv("./datasets/clickbait_data.csv")
df

<IPython.core.display.Javascript object>

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1
...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0
31996,"British Liberal Democrat Patsy Calton, 56, die...",0
31997,Drone smartphone app to help heart attack vict...,0
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0


In [4]:
df["title_lenght"] = df["headline"].apply(lambda x: len(x))   # characters of title
df["uppercased"] = df["headline"].apply(lambda x: len([l for l in x if l.isupper()]))
# bin_edges = [0,10,20,30,40,50,60,70,80,90,100,500]
# bin_labels = ['bin_1', 'bin_2', 'bin_3', 'bin_4', "bin_5", "bin_6","bin_7", "bin_8", "bin_9", "bin_10", "bin_11"]
# len_bins = pd.cut(df["title_lenght"], bins=bin_edges, labels=bin_labels, right=False)
# len_dummies = pd.get_dummies(len_bins)
# bin_edges2 = [0,3,6,9,12,15,18,21,24,60]
# bin_labels2 = ['ubin_1', 'ubin_2', 'ubin_3', 'ubin_4', "ubin_5", "ubin_6","ubin_7", "ubin_8", "ubin_9"]
# upper_bins = pd.cut(df["uppercased"], bins=bin_edges2, labels=bin_labels2, right=False)
# upper_dummies = pd.get_dummies(upper_bins)
# features = pd.concat([ upper_dummies,len_dummies], axis=1)

In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
df['sentiment_score'] = df['headline'].apply(lambda x: sia.polarity_scores(x)["compound"])
df

,headline,clickbait,title_lenght,uppercased,sentiment_score
0,Should I Get Bings,1,18,4,0.0000
1,Which TV Female Friend Group Do You Belong In,1,45,10,0.4939
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1,73,14,-0.5574
3,"This Vine Of New York On ""Celebrity Big Brothe...",1,67,12,0.6115
4,A Couple Did A Stunning Photo Shoot With Their...,1,100,18,-0.3400
...,...,...,...,...,...
31995,"To Make Female Hearts Flutter in Iraq, Throw a...",0,51,8,0.0000
31996,"British Liberal Democrat Patsy Calton, 56, die...",0,57,5,-0.6597
31997,Drone smartphone app to help heart attack vict...,0,75,1,-0.4019
31998,"Netanyahu Urges Pope Benedict, in Israel, to D...",0,58,7,-0.3400


In [6]:
df['sentiment_score'] = df['sentiment_score'].apply(lambda x: x**2)

In [7]:
import nltk
from nltk import sent_tokenize, word_tokenize

def coleman_liau_index(x):
    words = word_tokenize(x)
    words_count = len(words)
    sentences = sent_tokenize(x)
    sentences = len(sentences)
    letters = sum(len(letter) for word in words for letter in word)
    L = (letters / words_count) * 100
    S = (sentences / words_count) * 100
    return 0.0588 * L - 0.296 * S - 15.8
df["Readability"] = df["headline"].apply(coleman_liau_index)

In [10]:
import nltk #Natural Language tool kit -- this pacakge is quite a mess. Was poorly design and the documentation is not great
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
bow_vect_clickbait = CountVectorizer(max_features=1000)
# fit creates one entry for each different word seen

bow_vect_clickbait.fit(df['headline'])

CountVectorizer(max_features=1000)

In [42]:
import pickle
with open('bow_vect_clickbait.pkl', 'wb') as file:
    pickle.dump(bow_vect_clickbait, file)

In [24]:
vs = bow_vect_clickbait.transform(df['headline']).toarray()
vs = pd.DataFrame(X, columns=bow_vect_clickbait.get_feature_names_out())
vs

<IPython.core.display.Javascript object>

,000,00s,10,100,11,12,13,14,15,16,...,years,york,you,young,your,yourself,youtube,zealand,zimbabwe,zodiac
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
knn = MultinomialNB()
features = df[["title_lenght", "uppercased",]]
X = pd.concat([features, vs], axis=1)
y = df["clickbait"]
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

model = LogisticRegression(max_iter=2000)
model.fit(X, y)
model.score(X_test, y_test)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.980375

In [44]:
import pickle
with open('clickbait_model.pkl', 'wb') as file:
    pickle.dump(model, file)